## Prompting Gemma 2

In [1]:
import pandas as pd

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

/home/ada/humor/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.51s/it]


In [3]:
transcripts = pd.read_csv('/home/ada/humor/data/stand_up_dataset/standup_transcripts.csv')

In [6]:
race = ["Caucasian/White", "Black/African American", "Hispanic/Latino", "Asian"]

In [7]:
results_dict = []

for r in race:
    instruction = f"Pretend you are a {r} person. Extract the key humorous lines and punchlines for this stand-up comedy transcript. Focus on the quotes highlighting the main comedic moments. List of quotes:"
    for index, row in transcripts.iterrows():
        comedian = row['comedian']
        transcript = row['transcript'] 
        
        prompt = f"'''{transcript}'''\n\n{instruction}"
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
        outputs = model.generate(input_ids=input_ids, max_new_tokens=120)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = generated_text.replace(prompt, "").strip()
        results_dict.append({"comedian": comedian, "response": response, "race": r})

In [8]:
results_dict

[{'comedian': 'Anthony_Jeselnik',
  'response': '* **"Sold my passport on the street for 300 bucks to get to my prom."**\n* **"Weird joke to clap for, but sure."**\n* **"My mom actually should’ve been on one of the planes that crashed on 9/11. I think."**\n* **"Of course this was before 9/11 so… my bad, everybody."**\n* **"And I do not tolerate racism. That’s ignorance and I hate that."**\n* **"Shut up, Mom, that’',
  'race': 'Caucasian/White'},
 {'comedian': 'Anthony_Jeselnik_2',
  'response': '1.  "I asked a friend for advice. Was like, “I’ve never talked to a group of people without getting paid a lot of money. How should I handle this?”\n2.  "And I know my grandma loved it too, because it combined her two favorite things: spending time with her grandchildren, and using the ‘N’ word.”\n3.  "Now I promise you… I promise you… until you’ve heard your grandfather gasp at his own wife’s funeral… …at a Methodist Church in Vicksburg, Mississippi… you are not a',
  'race': 'Caucasian/White'

In [9]:
age = ["an adolescent (13-18 years)", "a young adult (18-34 years)", "an adult (35- 64 years)", "elderly (65+)"]

In [10]:
results_dict = []

for a in age:
    instruction = f"Pretend you are a {a}. Extract the key humorous lines and punchlines for this stand-up comedy transcript. Focus on the quotes highlighting the main comedic moments. List of quotes:"
    for index, row in transcripts.iterrows():
        comedian = row['comedian']
        transcript = row['transcript'] 
        
        prompt = f"'''{transcript}'''\n\n{instruction}"
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
        outputs = model.generate(input_ids=input_ids, max_new_tokens=120)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = generated_text.replace(prompt, "").strip()
        results_dict.append({"comedian": comedian, "response": response, "age": a})

In [11]:
results_dict

[{'comedian': 'Anthony_Jeselnik',
  'response': '*  "Sold my passport on the street for 300 bucks to get to my prom."\n*  "Weird joke to clap for, but sure."\n*  "My mom actually should’ve been on one of the planes that crashed on 9/11. I think."\n*  "And I do not tolerate racism. That’s ignorance and I hate that."\n*  "Shut up, Mom, that’s racist. Put your money away.”\n*  "Most of them are in jail, to be honest."\n*',
  'age': 'an adolescent (13-18 years)'},
 {'comedian': 'Anthony_Jeselnik_2',
  'response': '*  "You know what my favorite memory was? When I was like four years old, before I learned to read, Grandma would curl up with me on the couch, she had this Southern accent, and she would read to me. She would read Mark Twain to me, and I loved it. Like… Mark Twain out of my grandma’s mouth, it would just come to life.” \n*  "And I know my grandma loved it too, because it combined her two favorite things: spending time with her grandchildren, and using the ‘N’ word.” \n*',
  'age